This notebook uses code from `VHA.ipynb`. It's much more concise and doesn't do VQE; we just get the ground state so I can quickly analyze its properties. 

### Get ground state

In [19]:
import numpy as np
import scipy.linalg 

from tools.utils import * 
tol = 0.005

In [225]:
# Define lattice and model 
from openfermion.utils import HubbardSquareLattice
# HubbardSquareLattice parameters
x_n = 8
y_n = 1
n_dofs = 1 # 1 degree of freedom for spin, this might be wrong. Having only one dof means ordered=False. 
periodic = 0 # Not sure what this is, periodic boundary conditions?
spinless = 0 # Has spin

lattice = HubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

In [226]:
from openfermion.hamiltonians import FermiHubbardModel
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] # Not sure if this is right
interaction = [('onsite', (0, 0), 6., SpinPairs.DIFF)] # Not sure if this is right
potential = [(0, 0.)]
# potential = None
mag_field = 0. 
particle_hole_sym = False # Not sure if this is right

In [227]:
hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

In [228]:
# Get ground state and energy 
from openfermion import get_sparse_operator, get_ground_state
hub_sparse = get_sparse_operator(hubbard.hamiltonian())
genergy, gstate = get_ground_state(hub_sparse)
print("Ground state energy: ", genergy)

# w, v = scipy.sparse.linalg.eigsh(hub_sparse, k=200, which='SA')
# gstate=v[:, 199]

Ground state energy:  -5.581939204022831


### Analyze ground state

In [229]:
# Get average measurement of each qubit 
from cirq import measure_state_vector 
measurements = [measure_state_vector(gstate, range(2 * x_n * y_n))[0] for _ in range(10000)]
np.mean(measurements, axis=0)

array([0.3044, 0.2959, 0.3171, 0.3108, 0.3196, 0.3358, 0.3024, 0.3084,
       0.3173, 0.303 , 0.3261, 0.3276, 0.3164, 0.3191, 0.299 , 0.2971])

In Scalettar notes, magnetization $m$ is defined as $$m = \frac{\rho_\uparrow - \rho_\downarrow}{\rho_\uparrow + \rho_\downarrow}$$

In [230]:
# Calculate density 
sum(np.mean(measurements, axis=0)) / (x_n * y_n)

0.625

In [231]:
# Check spin balance 
up_spins = 0 
down_spins = 0
unequal = 0
for trial in measurements:
    up_spins += sum(trial[:x_n * y_n])
    down_spins += sum(trial[x_n * y_n:])
    if up_spins != down_spins:
        unequal += 1
print("m =", (up_spins - down_spins) / (up_spins + down_spins))

m = -0.00224


In [224]:
unequal

9398

In [123]:
pos = 0 
neg = 0 

for trial in measurements: 
    up_spins = sum(trial[:x_n * y_n])
    down_spins = sum(trial[x_n * y_n:])
    m = (up_spins - down_spins) / (up_spins + down_spins)
    if np.abs(m) > tol:  
        if m > 0: 
            pos += 1 
        else: 
            neg += 1

In [124]:
pos

520

In [125]:
neg

483